In [ ]:
# 只用执行一次 上传数据
import pandas as pd
import numpy as np
import pyodbc

df = pd.read_csv("House_Data.csv")
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
if 'sa2_coor' in df.columns:
    df = df.drop(columns=['sa2_coor'])


# Edit all the missing or NAN value
for col, dtype in df.dtypes.items():
    if dtype in ['float64', 'int64']:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # 非数字自动变为NaN

df = df.replace({np.nan: None})

# Connecting the Azure SQL database
conn_str = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=voiceai-sql-server.database.windows.net;"
    "DATABASE=RealEstateConvoAI;"
    "UID=lorraine;"
    "PWD=Test123!Temp;"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
    "Connection Timeout=30;"
)
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

columns = df.columns.tolist()
col_str = ','.join([f'[{col}]' for col in columns])
placeholders = ','.join(['?'] * len(columns))
insert_sql = f"INSERT INTO [House_Data] ({col_str}) VALUES ({placeholders})"
for col in df.columns:
    if df[col].dtype == 'object':
        print(f"{col}: max length = {df[col].astype(str).map(len).max()}")

try:
    cursor.fast_executemany = True
    cursor.executemany(insert_sql, df.values.tolist())
    conn.commit()
    print("All data has been upload onto Azure SQL")
except Exception as e:
    print("Upload wrong", e)
finally:
    cursor.close()
    conn.close()


In [ ]:
# Create a customer information table
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=voiceai-sql-server.database.windows.net;"
    "DATABASE=RealEstateConvoAI;"
    "UID=lorraine;"
    "PWD=Test123!Temp;"
)
cursor = conn.cursor()

# 写好 CREATE TABLE 语句（如果已存在先删掉，可选）
cursor.execute("IF OBJECT_ID('Customer_Table', 'U') IS NOT NULL DROP TABLE Customer_Table;")

create_sql = """
CREATE TABLE Customer_Table (
    id INT IDENTITY(1,1) PRIMARY KEY,
    name NVARCHAR(100),
    phone NVARCHAR(30),
    address NVARCHAR(300),
    created_at DATETIME DEFAULT GETDATE()
)
"""
cursor.execute(create_sql)
conn.commit()
cursor.close()
conn.close()

print("Customer_Table created")
